In [26]:
import os
from langchain_community.chat_models import ChatOllama
from langgraph.graph import END, StateGraph
from typing import Dict, TypedDict
from langchain.prompts import PromptTemplate

In [27]:
local_llm = 'elvee/hermes-2-pro-llama-3:8b-Q5_K_M'
llm = ChatOllama(model=local_llm, temperature=0)

Define tool

In [28]:
from langchain_core.tools import tool
from langchain_community.tools.tavily_search import TavilySearchResults

@tool
def web_search(query:str) -> str:
    """Search the web for query"""
    tool = TavilySearchResults(max_results=3,tavily_api_key=os.environ['TAVILY_API_KEY'])
    return str(tool.invoke(query))

In [29]:
print(web_search.name)
print(web_search.description)
print(web_search.args)

web_search
Search the web for query
{'query': {'title': 'Query', 'type': 'string'}}


Render tool description which prompts the llm

In [30]:
from langchain.tools.render import render_text_description

rendered_tools = render_text_description([web_search])
rendered_tools

'web_search(query: str) -> str - Search the web for query'

In [31]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = """You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool: 

""" + rendered_tools + """

Given the user input, return the name and input of the tool to use. 
Return your response as a JSON blob with 'name' and 'arguments' keys.
For example: 
{{'name': 'web_search', 'arguments': {{'arg1': "value_1"}}}}


"""

print(system_prompt)

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool: 

web_search(query: str) -> str - Search the web for query

Given the user input, return the name and input of the tool to use. 
Return your response as a JSON blob with 'name' and 'arguments' keys.
For example: 
{{'name': 'web_search', 'arguments': {{'arg1': "value_1"}}}}





In [32]:
from langchain_core.output_parsers import JsonOutputParser


chain = prompt | llm | JsonOutputParser()
chain.invoke({"input": "Search Lawrence Wong"})

{'name': 'web_search', 'arguments': {'query': 'Lawrence Wong'}}

In [33]:
from operator import itemgetter

def search(query):
    search_chain = prompt | llm | JsonOutputParser() | itemgetter("arguments") | web_search
    return search_chain.invoke({"input": query})

search("Lawrence Wong")


'[{\'url\': \'https://en.wikipedia.org/wiki/Lawrence_Wong\', \'content\': "Lawrence Wong Shyun Tsai [b] (born 18 December 1972) is a Singaporean politician, economist and former civil servant who has been serving as the fourth prime minister of Singapore since 2024 and the minister for finance since 2021. A member of the governing People\'s Action Party, he has been the Member of Parliament (MP) representing the ..."}, {\'url\': \'https://www.reuters.com/world/asia-pacific/singapores-new-leader-lawrence-wong-2024-05-15/\', \'content\': \'Lawrence Wong, 51, will take over as prime minister of Singapore on Wednesday, becoming the fourth leader of the Asian financial hub since its independence in 1965.\'}, {\'url\': \'https://www.britannica.com/biography/Lawrence-Wong\', \'content\': "Lawrence Wong (born December 18, 1972, Singapore) is the current and fourth prime minister of Singapore (2024-). An economist by training, he served in several government ministries before becoming deputy pr

In [34]:
summariser_system_prompt = """

The JSON object you are given contains some search results. 
Return a summary of the search results. 

"""

def search_summarise(query):
    search_result = search(query)
    summariser_prompt = ChatPromptTemplate.from_messages(
        [("system", summariser_system_prompt + search_result.replace("{", "[").replace("}", "]"))]
    )
    summariser_chain = summariser_prompt | llm 
    return summariser_chain.invoke({}).content

search_summarise("Speakers for ATX Summit")

'The ATxSummit, organized by Infocomm Media Development Authority of Singapore, will take place from 30-31 May 2024 at Capella Singapore. This event includes an invitation-only Plenary conference covering topics such as Artificial Intelligence, Governance and Safety, Quantum Computing, Sustainability, and more. The Asia Tech x Singapore event features a range of co-located events for participants to network and stay updated on the latest tech trends.\n\nRegistration is now open for ATxSG 2024, which will host over 25,000 hybrid participants, 1000+ sponsors and exhibitors, and 420+ speakers. The event focuses on sustainability and is organized by Infocomm Media Development Authority (IMDA) and Informa Tech, with support from the Singapore Tourism Board (STB).'